In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets
from PIL import Image
import pandas as pd
import os

# 定义残差块
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# 定义截取的 AlexNet 部分
class AlexNetPartial(nn.Module):
    def __init__(self):
        super(AlexNetPartial, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),  # 新增的卷积层
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),  # 新增的卷积层
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.features(x)
        return x

# 定义部分 ResNet（仅包含残差块）
class ResNetPartial(nn.Module):
    def __init__(self, block, num_blocks):
        super(ResNetPartial, self).__init__()
        self.in_planes = 128  # 确保与 AlexNet 的输出通道数一致

        self.layer1 = self._make_layer(block, 128, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block, 256, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 512, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        return x

# 定义组合网络
class Net(nn.Module):
    def __init__(self):
        super( Net, self).__init__()
        self.alexnet_partial = AlexNetPartial()
        self.resnet_partial = ResNetPartial(BasicBlock, [2, 2, 2, 2])
        self.fc = nn.Linear(512, 5)  # 输出5个类别

    def forward(self, x):
        x = self.alexnet_partial(x)
        x = self.resnet_partial(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x




AttributeError: partially initialized module 'torch._dynamo' has no attribute 'trace_rules' (most likely due to a circular import)

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, csv_f, img_folder, transform, s_num=4000):
        self.img_labels = pd.read_csv(csv_f)[:s_num]
        self.img_folder = img_folder
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_name = str(self.img_labels.iloc[idx, 0])
        label = self.img_labels.iloc[idx, 1]
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


pre = transforms.Compose([transforms.RandomResizedCrop(224),
                          transforms.RandomHorizontalFlip(),
                          transforms.RandomRotation(15),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                          ])

csv_f = '../data/cassava-leaf-disease-classification/train.csv'
img_folder = '../data/cassava-leaf-disease-classification/train_images'
data = CassavaDataset(csv_f, img_folder, transform=pre, s_num=4000)

train_size = int(0.9 * len(data))
train_dataset, test_dataset = random_split(data, [train_size, (len(data) - train_size)])

batch_size = 32

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
num_classes = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



net = Net()
net.to(device)
loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)
t_steps = len(trainloader)
test_num = len(test_dataset)

for epoch in range(5):
    net.train()
    running_loss = 0.0
    for step, data in enumerate(trainloader, start=0):
        imgs, labels = data
        rate = ((step + 1) / t_steps)
        optimizer.zero_grad()
        outputs = net(imgs.to(device))
        loss = loss_f(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        a = '*' * int(rate * 50)
        b = '.' * int((1 - rate) * 50)
        print('\r train loss: {:^3.0f}%[ {} -> {} ]{:.3f}'.format(int(rate * 100), a, b, loss), end='')
    print()

    net.eval()
    r = 0.0
    best_R = 0.0
    with torch.no_grad():
        for data_test in testloader:
            t_img, t_lbl = data_test
            outputs = net(t_img.to(device))
            pred_y = torch.max(outputs, dim=1)[1]
            r += (pred_y == t_lbl.to(device)).sum().item()
        acc_R = r / test_num
        if acc_R > best_R:
            best_R = acc_R
            torch.save(net.state_dict(), '../path/Net_best.pth')
        print('[epoch %d] train_loss: %.3f    test_accuracy: %.3f' % (epoch + 1, running_loss, acc_R))

    print('finished')